In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.nn import Parameter
import torch.nn.functional as F
from utils import match_loss, regularization, row_normalize_tensor
import deeprobust.graph.utils as utils
from copy import deepcopy
import numpy as np
from tqdm import tqdm
from models.gcn import GCN
from models.sgc import SGC
from models.sgc_multi import SGC as SGC1
from models.parametrized_adj import PGE
import scipy.sparse as sp
from torch_sparse import SparseTensor
import  torch, os
import  numpy as np
from    subgraph_data_processing import Subgraphs,Subgraphs_syn
import  scipy.stats
from    torch.utils.data import DataLoader
from    torch.optim import lr_scheduler
import  random, sys, pickle
import  argparse

import networkx as nx
import numpy as np
from scipy.special import comb
from itertools import combinations 
import networkx.algorithms.isomorphism as iso
from tqdm import tqdm
import dgl

from meta import Meta
import time
import copy
import psutil
from memory_profiler import memory_usage

os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [2]:
import argparse
parser = argparse.ArgumentParser()
parser.add_argument('--gpu_id', type=int, default=0, help='gpu id')
parser.add_argument('--dataset', type=str, default='cora')
parser.add_argument('--dis_metric', type=str, default='ours')
parser.add_argument('--epochs', type=int, default=2000)
parser.add_argument('--nlayers', type=int, default=3)
parser.add_argument('--hidden', type=int, default=256)
parser.add_argument('--lr_adj', type=float, default=0.01)
parser.add_argument('--lr_feat', type=float, default=0.01)
parser.add_argument('--lr_model', type=float, default=0.01)
parser.add_argument('--weight_decay', type=float, default=0.0)
parser.add_argument('--dropout', type=float, default=0.0)
parser.add_argument('--normalize_features', type=bool, default=True)
parser.add_argument('--keep_ratio', type=float, default=1.0)
parser.add_argument('--reduction_rate', type=float, default=1)
parser.add_argument('--seed', type=int, default=15, help='Random seed.')
parser.add_argument('--alpha', type=float, default=0, help='regularization term.')
parser.add_argument('--debug', type=int, default=0)
parser.add_argument('--sgc', type=int, default=1)
parser.add_argument('--inner', type=int, default=0)
parser.add_argument('--outer', type=int, default=20)
parser.add_argument('--save', type=int, default=0)
parser.add_argument('--one_step', type=int, default=0)

parser.add_argument('--epoch', type=int, help='epoch number', default=10)
parser.add_argument('--n_way', type=int, help='n way', default=3)
# parser.add_argument('--k_spt', type=int, help='k shot for support set', default=3)
# parser.add_argument('--k_qry', type=int, help='k shot for query set', default=24)
parser.add_argument('--k_spt', type=int, help='k shot for support set', default=1)
parser.add_argument('--k_qry', type=int, help='k shot for query set', default=3)
parser.add_argument('--task_num', type=int, help='meta batch size, namely task num', default=32)
parser.add_argument('--meta_lr', type=float, help='meta-level outer learning rate', default=1e-3)
parser.add_argument('--update_lr', type=float, help='task-level inner update learning rate', default=1e-2)
parser.add_argument('--update_step', type=int, help='task-level inner update steps', default=10)
parser.add_argument('--update_step_test', type=int, help='update steps for finetunning', default=20)
parser.add_argument('--input_dim', type=int, help='input feature dim', default=1)
parser.add_argument('--hidden_dim', type=int, help='hidden dim', default=256)
parser.add_argument('--attention_size', type=int, help='dim of attention_size', default=32)
parser.add_argument("--data_dir", default="D:\pythonProject\python_file\Graph_DD\G-Meta-master\arxiv", type=str, required=False, help="The input data dir.")
parser.add_argument("--no_finetune", default=True, type=str, required=False, help="no finetune mode.")
parser.add_argument("--task_setup", default='Disjoint', type=str, required=False, help="Select from Disjoint or Shared Setup. For Disjoint-Label, single/multiple graphs are both considered.")
parser.add_argument("--method", default='G-Meta', type=str, required=False, help="Use G-Meta")
parser.add_argument('--task_n', type=int, help='task number', default=1)
parser.add_argument("--task_mode", default='False', type=str, required=False, help="For Evaluating on Tasks")
parser.add_argument("--val_result_report_steps", default=100, type=int, required=False, help="validation report")
parser.add_argument("--train_result_report_steps", default=200, type=int, required=False, help="training report")
parser.add_argument("--num_workers", default=0, type=int, required=False, help="num of workers")
parser.add_argument("--batchsz", default=10000, type=int, required=False, help="batch size")
parser.add_argument("--link_pred_mode", default='False', type=str, required=False, help="For Link Prediction")
parser.add_argument("--h", default=2, type=int, required=False, help="neighborhood size")
parser.add_argument('--sample_nodes', type=int, help='sample nodes if above this number of nodes', default=300)
args = parser.parse_args([])

In [3]:
import csv
from collections import Counter

def generate_labels_from_csv(file_path):
    with open(file_path, 'r') as csv_file:
        reader = csv.DictReader(csv_file)
        data = list(reader)

    counter = Counter()  # 使用 Counter 对标签进行计数
    num_class_dict = {}  # 存储每个类别需要生成的节点数
    n = len(data)  # 数据总数

    # 统计每个类别的出现次数
    for row in data:
        label = int(row['label'])
        counter[label] += 1

    # 根据标签出现次数进行排序
    sorted_counter = sorted(counter.items(), key=lambda x: x[1])
    sum_ = 0
    labels_syn = []  # 合成标签列表
    syn_class_indices = {}  # 记录每个类别的索引范围
#     ratio = 0.08
    ratio = 0.01
    # 计算每个类别需要生成的节点数，并生成合成标签
    for ix, (c, num) in enumerate(sorted_counter):

        if ix == len(sorted_counter) - 1:
            num_class_dict[c] = int(n * ratio) - sum_  
            # 如果是最后一个类别，那么直接计算需要生成的节点数，并将这个类别的合成标签添加到 labels_syn 列表中
            syn_class_indices[c] = [len(labels_syn), len(labels_syn) + num_class_dict[c]]
            labels_syn += [c] * num_class_dict[c]

        else:
            num_class_dict[c] = max(int(num * ratio), 1)  # 其他类别的节点数
            # 如果不是最后一个类别，那么先计算需要生成的节点数，然后将这个类别的合成标签添加到 labels_syn 列表中。
            sum_ += num_class_dict[c]
            syn_class_indices[c] = [len(labels_syn), len(labels_syn) + num_class_dict[c]]
            labels_syn += [c] * num_class_dict[c]

    return n * ratio,labels_syn


In [4]:
def loadCSV():

    dictLabels = {}

    # dictLabels（标签到子图的映射）、

    with open(r"../DATA/arxiv/train.csv") as csvfile:
        csvreader = csv.reader(csvfile, delimiter=',')
        next(csvreader, None)  # skip (filename, label)
        for i, row in enumerate(csvreader):
            filename = row[1]
            g_idx = int(filename.split('_')[0])
            label = row[2]
            # append filename to current label

            if label in dictLabels.keys():
                dictLabels[label].append(filename)
            else:
                dictLabels[label] = [filename]

    return dictLabels

In [10]:
import numpy as np

def get_sub_adj_feat(features, labels_syn, label_dict):
    idx_selected = []

    from collections import Counter
    counter = Counter(labels_syn.cpu().numpy())
    selected_features = []
    for c in counter.keys():
        class_label = str(c)
        if class_label in label_dict:
            node_indices = [label_dict[class_label].index(node) for node in label_dict[class_label]]
            num_features = counter[c]
            selected_indices = np.random.choice(node_indices, size=num_features, replace=False)

            idx_selected.extend(selected_indices)

            selected_features.extend(features[selected_indices])

    selected_features = np.array(selected_features)

    return selected_features


In [11]:
feat = np.load( r'../DATA/arxiv/features.npy', allow_pickle = True)

In [12]:
feat

array([[-0.057943, -0.05253 , -0.072603, ...,  0.173364, -0.172796,
        -0.140059],
       [-0.1245  , -0.070665, -0.325202, ...,  0.068524, -0.372111,
        -0.301036],
       [-0.080242, -0.023328, -0.183787, ...,  0.109919,  0.117589,
        -0.139883],
       ...,
       [-0.22053 , -0.036568, -0.402199, ...,  0.11336 , -0.161393,
        -0.145171],
       [-0.138236,  0.040885, -0.251811, ..., -0.08929 , -0.041253,
        -0.376132],
       [-0.029875,  0.268417, -0.161124, ...,  0.120807,  0.077647,
        -0.091018]], dtype=float32)

In [13]:
n,labels_syn= generate_labels_from_csv(r'../DATA/arxiv/train.csv')

In [14]:
# 计算需要生成的synthetic graph的节点数
n = int(n)
# 获取特征维度
d = 128
device='cuda:4'
# 初始化synthetic graph的节点数和特征
nnodes_syn = n



# 初始化PGE模型
pge = PGE(nfeat=d, nnodes=n, device=device, args=args).to(device)

# 生成synthetic graph的标签
labels_syn = torch.LongTensor(labels_syn).to(device)

dictLabels = loadCSV()
feat_syn = torch.from_numpy(get_sub_adj_feat(feat, labels_syn, dictLabels)).to(device)

In [15]:


# # 重置synthetic graph的特征
# reset_parameters()

# 初始化特征优化器和PGE模型优化器
optimizer_feat = torch.optim.Adam([feat_syn], lr=0.01)
optimizer_pge = torch.optim.Adam(pge.parameters(), lr=0.01)

# 打印synthetic graph的节点数和特征维度
print('adj_syn:', (n,n), 'feat_syn:', feat_syn.shape)

adj_syn: (1006, 1006) feat_syn: torch.Size([1006, 128])


In [16]:
from collections import Counter

counter = Counter(labels_syn.tolist())

print(counter)

Counter({24: 231, 28: 214, 8: 62, 4: 58, 5: 49, 27: 48, 2: 48, 26: 46, 9: 28, 31: 28, 23: 28, 13: 23, 39: 20, 20: 20, 3: 20, 22: 19, 25: 12, 33: 12, 18: 7, 1: 6, 17: 5, 0: 5, 7: 5, 15: 4, 32: 4, 29: 4})


In [17]:
adj_syn = pge(feat_syn)
# adj_syn_norm = utils.normalize_adj_tensor(adj_syn, sparse=False)

In [18]:
adj_syn[adj_syn < 0.5] = 0
adj_syn[adj_syn >= 0.5] = 1

In [19]:

adj_syn_norm_sp = sp.csr_matrix(adj_syn.cpu().detach().numpy())
g = dgl.from_scipy(adj_syn_norm_sp) # 生成的合成图太过密集，导致二跳子图约等于全图


In [20]:
g

Graph(num_nodes=1006, num_edges=651330,
      ndata_schemes={}
      edata_schemes={})

In [21]:
# 将labels_syn转换为dictLabels形式的字典
dict_labels = {f"0_{i}": labels_syn[i].item() for i in range(len(labels_syn))}

In [22]:
db_val = Subgraphs_syn('syn', dict_labels, n_way=args.n_way, k_shot=args.k_spt,k_query=args.k_qry, batchsz=100, args = args, adjs = g, h = args.h, labels = labels_syn)

shuffle DB :syn, b:100, 3-way, 1-shot, 3-query, 2-hops


In [23]:
for step, (x_spt, y_spt, x_qry, y_qry, c_spt, c_qry, n_spt, n_qry, g_spt, g_qry) in enumerate(db_val): # 处理每个任务
    print('step:', step, '\n')
    print('x_spt:', x_spt,'\n')

step: 0 

x_spt: Graph(num_nodes=902, num_edges=169856,
      ndata_schemes={'_ID': Scheme(shape=(), dtype=torch.int64)}
      edata_schemes={'_ID': Scheme(shape=(), dtype=torch.int64)}) 

step: 1 

x_spt: Graph(num_nodes=902, num_edges=178680,
      ndata_schemes={'_ID': Scheme(shape=(), dtype=torch.int64)}
      edata_schemes={'_ID': Scheme(shape=(), dtype=torch.int64)}) 

step: 2 

x_spt: Graph(num_nodes=903, num_edges=172894,
      ndata_schemes={'_ID': Scheme(shape=(), dtype=torch.int64)}
      edata_schemes={'_ID': Scheme(shape=(), dtype=torch.int64)}) 



KeyboardInterrupt: 

In [16]:
def collate(samples):
        graphs_spt, labels_spt, graph_qry, labels_qry, center_spt, center_qry, nodeidx_spt, nodeidx_qry, support_graph_idx, query_graph_idx = map(list, zip(*samples))

        return graphs_spt, labels_spt, graph_qry, labels_qry, center_spt, center_qry, nodeidx_spt, nodeidx_qry, support_graph_idx, query_graph_idx


In [17]:
db = DataLoader(db_val, args.task_num, shuffle=True, num_workers=args.num_workers, pin_memory=True, collate_fn = collate)

In [18]:
num_steps = len(db)
print("Number of steps:", num_steps)


Number of steps: 4


In [19]:
for step, (x_spt, y_spt, x_qry, y_qry, c_spt, c_qry, n_spt, n_qry, g_spt, g_qry) in enumerate(db):
    # 对于合成图而言，每次采样出来的子集接近原图大小，因此通过--sample_nodes进行限制，当然这种办法并不能加快采样过程，只是能缓解训练batch过大的情况


begin1
Graph(num_nodes=301, num_edges=48366,
      ndata_schemes={'_ID': Scheme(shape=(), dtype=torch.int64)}
      edata_schemes={'_ID': Scheme(shape=(), dtype=torch.int64)})
Graph(num_nodes=300, num_edges=45310,
      ndata_schemes={'_ID': Scheme(shape=(), dtype=torch.int64)}
      edata_schemes={'_ID': Scheme(shape=(), dtype=torch.int64)})
Graph(num_nodes=300, num_edges=46704,
      ndata_schemes={'_ID': Scheme(shape=(), dtype=torch.int64)}
      edata_schemes={'_ID': Scheme(shape=(), dtype=torch.int64)})
end1
Graph(num_nodes=300, num_edges=50214,
      ndata_schemes={'_ID': Scheme(shape=(), dtype=torch.int64)}
      edata_schemes={'_ID': Scheme(shape=(), dtype=torch.int64)})
Graph(num_nodes=300, num_edges=44544,
      ndata_schemes={'_ID': Scheme(shape=(), dtype=torch.int64)}
      edata_schemes={'_ID': Scheme(shape=(), dtype=torch.int64)})
Graph(num_nodes=300, num_edges=51886,
      ndata_schemes={'_ID': Scheme(shape=(), dtype=torch.int64)}
      edata_schemes={'_ID': Scheme(shape

Graph(num_nodes=300, num_edges=44232,
      ndata_schemes={'_ID': Scheme(shape=(), dtype=torch.int64)}
      edata_schemes={'_ID': Scheme(shape=(), dtype=torch.int64)})
Graph(num_nodes=301, num_edges=46484,
      ndata_schemes={'_ID': Scheme(shape=(), dtype=torch.int64)}
      edata_schemes={'_ID': Scheme(shape=(), dtype=torch.int64)})
end1
Graph(num_nodes=300, num_edges=42968,
      ndata_schemes={'_ID': Scheme(shape=(), dtype=torch.int64)}
      edata_schemes={'_ID': Scheme(shape=(), dtype=torch.int64)})
Graph(num_nodes=301, num_edges=44930,
      ndata_schemes={'_ID': Scheme(shape=(), dtype=torch.int64)}
      edata_schemes={'_ID': Scheme(shape=(), dtype=torch.int64)})
Graph(num_nodes=301, num_edges=45726,
      ndata_schemes={'_ID': Scheme(shape=(), dtype=torch.int64)}
      edata_schemes={'_ID': Scheme(shape=(), dtype=torch.int64)})
Graph(num_nodes=301, num_edges=47426,
      ndata_schemes={'_ID': Scheme(shape=(), dtype=torch.int64)}
      edata_schemes={'_ID': Scheme(shape=(), dt

Graph(num_nodes=301, num_edges=46164,
      ndata_schemes={'_ID': Scheme(shape=(), dtype=torch.int64)}
      edata_schemes={'_ID': Scheme(shape=(), dtype=torch.int64)})
begin1
Graph(num_nodes=301, num_edges=44540,
      ndata_schemes={'_ID': Scheme(shape=(), dtype=torch.int64)}
      edata_schemes={'_ID': Scheme(shape=(), dtype=torch.int64)})
Graph(num_nodes=301, num_edges=46002,
      ndata_schemes={'_ID': Scheme(shape=(), dtype=torch.int64)}
      edata_schemes={'_ID': Scheme(shape=(), dtype=torch.int64)})
Graph(num_nodes=300, num_edges=46432,
      ndata_schemes={'_ID': Scheme(shape=(), dtype=torch.int64)}
      edata_schemes={'_ID': Scheme(shape=(), dtype=torch.int64)})
end1
Graph(num_nodes=301, num_edges=48686,
      ndata_schemes={'_ID': Scheme(shape=(), dtype=torch.int64)}
      edata_schemes={'_ID': Scheme(shape=(), dtype=torch.int64)})
Graph(num_nodes=301, num_edges=47472,
      ndata_schemes={'_ID': Scheme(shape=(), dtype=torch.int64)}
      edata_schemes={'_ID': Scheme(shape

Traceback (most recent call last):
  File "C:\Users\pc\AppData\Roaming\Python\Python39\site-packages\IPython\core\interactiveshell.py", line 3398, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\pc\AppData\Local\Temp\ipykernel_22724\3303405575.py", line 1, in <cell line: 1>
    for step, (x_spt, y_spt, x_qry, y_qry, c_spt, c_qry, n_spt, n_qry, g_spt, g_qry) in enumerate(db):
  File "C:\Users\pc\AppData\Roaming\Python\Python39\site-packages\torch\utils\data\dataloader.py", line 634, in __next__
    data = self._next_data()
  File "C:\Users\pc\AppData\Roaming\Python\Python39\site-packages\torch\utils\data\dataloader.py", line 678, in _next_data
    data = self._dataset_fetcher.fetch(index)  # may raise StopIteration
  File "C:\Users\pc\AppData\Roaming\Python\Python39\site-packages\torch\utils\data\_utils\fetch.py", line 51, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "C:\Users\pc\AppData\Roaming\Python\Python39\site-